In [246]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy import stats

$X= ``Número\ de\ gols\ do\ time\ de\ casa" \sim Poisson(\lambda_x)\\$
$Y= ``Número\ de\ gols\ do\ time\ visitante" \sim Poisson(\lambda_x)$

$$P(X=x)=\frac{\lambda^{x}_{x}e^{-\lambda_x}}{x!}\qquad P(Y=y)=\frac{\lambda^{y}_{y}e^{-\lambda_y}}{y!}\\$$

$$P(X=x, Y=y)=P(X=x)P(Y=y)=\frac{\lambda_{x}^{x}\lambda_{y}^{y}e^{-(\lambda_x+\lambda_y)}}{x!y!}$$

In [247]:
#poisson.pmf(k, mu, loc)

def gerar_prob(timex_str, timey_str, lambdax, lambday):
  timex = []
  for k in range(0, 11):
    timex.append(stats.poisson.pmf(k, lambdax))

  timey = []
  for k in range(0, 11):
    timey.append(stats.poisson.pmf(k, lambday))

  df_prob1 = pd.DataFrame({timex_str: timex, timey_str: timey})

  df_prob2 = pd.DataFrame()
  x_win_percent = []
  y_win_percent = []

  for i in range(1, len(df_prob1) - 1):
      x_win_percent.append(df_prob1['Cuiaba'][i + 1] * sum(df_prob1['Cruzeiro'][df_prob1['Cruzeiro'].index < i]))
      y_win_percent.append(df_prob1['Cruzeiro'][i + 1] * sum(df_prob1['Cuiaba'][df_prob1['Cuiaba'].index < i]))

  df_prob2[f'{timex_str} Win% if score'] = x_win_percent
  df_prob2[f'{timey_str} Win% if score'] = y_win_percent
  df_prob2.index = pd.RangeIndex(1,10)

  win_cuiaba = sum(df_prob2[f'{timex_str} Win% if score'])
  win_cruzeiro = sum(df_prob2[f'{timey_str} Win% if score'])

  df_prob3 = pd.DataFrame({f'{timex_str} Win%': [win_cuiaba], f'{timey_str} Win%': [win_cruzeiro]})

  df_final = pd.concat([df_prob1, df_prob2, df_prob3], axis=1)
  df_final.loc[0, 'empate'] = 1-(df_final['Cruzeiro Win%'][0] + df_final['Cuiaba Win%'][0])
  df_final.loc[1, 'empate'] = 1/df_final.loc[0, 'empate']
  df_final.loc[1, f'{timex_str} Win%'] = 1/df_final.loc[0, f'{timex_str} Win%']
  df_final.loc[1, f'{timey_str} Win%'] = 1/df_final.loc[0, f'{timey_str} Win%']

  return df_final

# MODELO 1
xG esperado para a partida

In [248]:
xg_cruzeiro_1410 = (0.14+3.87+1.63+1.56+1.74+0.35+0.66+0.58+0.88+0.97)/10
xg_cuiaba_1410 = (9.72+1.80+0.90+1.18+2.14+0.42+0.91+0.76)/10

In [249]:
df_1410 = gerar_prob('Cuiaba', 'Cruzeiro', xg_cuiaba_1410, xg_cruzeiro_1410)
df_1410.to_excel('dados/previsoes/previsao1410-cuiabavscruzeiro.xlsx')

In [250]:
df_1410

,Cuiaba,Cruzeiro,Cuiaba Win% if score,Cruzeiro Win% if score,Cuiaba Win%,Cruzeiro Win%,empate
0,0.168133,2.899636e-01,NaN,NaN,0.276429,0.109111,0.614460
1,0.299781,3.589749e-01,0.077494,3.736007e-02,3.617560,9.164972,1.627447
2,0.267255,2.222055e-01,0.103076,4.290622e-02,NaN,NaN,NaN
3,0.158838,9.169679e-02,0.061679,2.086421e-02,NaN,NaN,NaN
4,0.070802,2.838016e-02,0.024310,6.282125e-03,NaN,NaN,NaN
5,0.025248,7.026926e-03,0.007437,1.398867e-03,NaN,NaN,NaN
6,0.007503,1.449889e-03,0.001908,2.538738e-04,NaN,NaN,NaN
7,0.001911,2.564233e-04,0.000426,3.958470e-05,NaN,NaN,NaN
8,0.000426,3.968150e-05,0.000084,5.455527e-06,NaN,NaN,NaN
9,0.000084,5.458411e-06,0.000015,6.756821e-07,NaN,NaN,NaN


# MODELO 2
ATT Rating = (Gols Feitos no campeonato/Partidas Jogadas)/Média de gols da liga\
DEF Rating = (Gols Sofridos no campeonato/Partidas Jogadas)/Média de gols da liga\
EXPECTED GOALS = ATTxDEFxAVGLeague

In [251]:
tabela = pd.read_excel('dados/tabela_brasileirao(rodada-26).xlsx')

In [252]:
tabela

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Last 5,xGConv,xGAConv,GF/MP,GA/MP
0,1,Botafogo (RJ),26,17,4,5,42,16,26,55,2.12,34.7,30.6,4.1,0.16,L L L D W,7.3,-14.6,1.615385,0.615385
1,2,Bragantino,26,12,10,4,36,23,13,46,1.77,37.7,29.4,8.3,0.32,D W W W D,-1.7,-6.4,1.384615,0.884615
2,3,Grêmio,26,13,5,8,42,35,7,44,1.69,34.3,37.4,-3.2,-0.12,L D W D L,7.7,-2.4,1.615385,1.346154
3,4,Palmeiras,26,12,8,6,40,22,18,44,1.69,37.9,27.4,10.5,0.41,D W L L L,2.1,-5.4,1.538462,0.846154
4,5,Flamengo,26,12,8,6,38,31,7,44,1.69,35.2,34.5,0.7,0.03,W L D W D,2.8,-3.5,1.461538,1.192308
5,6,Fortaleza,26,12,6,8,33,26,7,42,1.62,39.8,29.4,10.5,0.40,L W W D W,-6.8,-3.4,1.269231,1.000000
6,7,Fluminense,26,12,5,9,34,31,3,41,1.58,35.2,33.1,2.1,0.08,W L W L L,-1.2,-2.1,1.307692,1.192308
7,8,Ath Paranaense,26,11,8,7,38,30,8,41,1.58,37.1,29.5,7.7,0.29,D W W L D,0.9,0.5,1.461538,1.153846
8,9,Atlético Mineiro,26,11,7,8,30,21,9,40,1.54,31.4,26.6,4.8,0.19,D W W W L,-1.4,-5.6,1.153846,0.807692
9,10,São Paulo,26,9,8,9,31,27,4,35,1.35,36.3,27.1,9.1,0.35,L L W W D,-5.3,-0.1,1.192308,1.038462


In [253]:
#ATT Rating = (Gols Feitos no campeonato/Partidas Jogadas)/gols_liga
#DEF Rating = (Gols Sofridos no campeonato/Partidas Jogadas)/gols_liga
#EXPECTED GOALS = ATTxDEFxAVGLeague

gols_feitos = tabela['GF']/tabela['MP']
gols_tomados = tabela['GA']/tabela['MP']
gols_liga = tabela['GF']/tabela['MP'].mean()

att_rating = gols_feitos/gols_liga
def_rating = gols_tomados/gols_liga

tabela['expG'] = att_rating*def_rating*gols_liga

In [254]:
df_1410_modelo2 = gerar_prob('Cuiaba', 'Cruzeiro', float(tabela[tabela['Squad'] == 'Cuiabá']['expG']), float(tabela[tabela['Squad'] == 'Cruzeiro']['expG']))
df_1410_modelo2.to_excel('dados/previsoes/previsao1410-cuiabavscruzeiro_modelo2.xlsx')

In [255]:
df_1410_modelo2

,Cuiaba,Cruzeiro,Cuiaba Win% if score,Cruzeiro Win% if score,Cuiaba Win%,Cruzeiro Win%,empate
0,2.884597e-01,4.018486e-01,NaN,NaN,0.196215,0.093469,0.710316
1,3.586131e-01,3.663573e-01,8.957762e-02,4.817285e-02,5.096462,10.698693,1.407824
2,2.229139e-01,1.670003e-01,7.096341e-02,3.283912e-02,NaN,NaN,NaN
3,9.237551e-02,5.075028e-02,2.685006e-02,1.006314e-02,NaN,NaN,NaN
4,2.871031e-02,1.156700e-02,7.038281e-03,2.029700e-03,NaN,NaN,NaN
5,7.138531e-03,2.109081e-03,1.475441e-03,3.176069e-04,NaN,NaN,NaN
6,1.479104e-03,3.204679e-04,2.625923e-04,4.166306e-05,NaN,NaN,NaN
7,2.626888e-04,4.173773e-05,4.081992e-05,4.754958e-06,NaN,NaN,NaN
8,4.082184e-05,4.756432e-06,5.638827e-06,4.817932e-07,NaN,NaN,NaN
9,5.638857e-06,4.818160e-07,7.010223e-07,4.392592e-08,NaN,NaN,NaN


# MODELO 3
lambda = xg(total) + xg(casa) + gols marcados(total) + gols marcados(casa) + xG(escalação)/5

In [256]:
historico = pd.read_excel('dados/historico_casa(2023-10-08).xlsx')

In [257]:
historico

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Blocks,SCA,GCA,TklW,Def 3rd,Mid 3rd,Att 3rd,Int,Err,Equipe
0,2023-04-15,18:30,Série A,Matchweek 1,Sat,Home,W,2,1,São Paulo,...,9,19,3,6,4,4,1,17,0,Botafogo-RJ
1,2023-05-07,18:30,Série A,Matchweek 4,Sun,Home,W,2,0,Atlético Mineiro,...,6,38,3,14,10,12,0,13,1,Botafogo-RJ
2,2023-05-11,19:30,Série A,Matchweek 5,Thu,Home,W,3,0,Corinthians,...,8,27,5,5,4,5,1,11,0,Botafogo-RJ
3,2023-05-20,18:30,Série A,Matchweek 7,Sat,Home,W,1,0,Fluminense,...,7,33,1,12,9,8,2,9,0,Botafogo-RJ
4,2023-05-28,19:00,Série A,Matchweek 8,Sun,Home,W,2,0,América (MG),...,9,34,4,4,5,6,0,11,0,Botafogo-RJ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,2023-08-13,16:00,Série A,Matchweek 19,Sun,Home,L,0,1,Goiás,...,9,29,0,11,10,6,3,12,0,America-MG
255,2023-08-27,16:00,Série A,Matchweek 21,Sun,Home,W,2,1,São Paulo,...,4,17,4,13,8,7,2,8,0,America-MG
256,2023-09-03,18:30,Série A,Matchweek 22,Sun,Home,W,2,0,Santos,...,7,21,4,7,6,5,2,8,0,America-MG
257,2023-09-19,21:30,Série A,Matchweek 24,Tue,Home,L,0,2,Bragantino,...,6,13,0,11,9,5,1,15,2,America-MG


In [258]:
cruzeiro_xgfora = historico[historico['Opponent'] == 'Cruzeiro']['xG'].mean()
cruzeiro_xgtotal = historico[(historico['Opponent'] == 'Cruzeiro') | (historico['Equipe'] == 'Cruzeiro')]['xG'].mean()
cruzeiro_golsfora = historico[historico['Opponent'] == 'Cruzeiro']['GA'].mean()
cruzeiro_golstotal = (cruzeiro_golsfora + historico[historico['Equipe'] == 'Cruzeiro']['GF'].mean())/2
xg_cruzeiro_1410

cuiaba_xgcasa = historico[historico['Equipe'] == 'Cuiaba']['xG'].mean()
cuiaba_xgtotal = historico[(historico['Opponent'] == 'Cuiabá') | (historico['Equipe'] == 'Cuiaba')]['xG'].mean()
cuiaba_golscasa = historico[historico['Equipe'] == 'Cuiaba']['GF'].mean()
cuiaba_golstotal = (cuiaba_golscasa + historico[historico['Opponent'] == 'Cuiabá']['GA'].mean())/2
xg_cuiaba_1410


1.7830000000000006

In [259]:
lambda_x =(cuiaba_xgcasa + cuiaba_xgtotal + cuiaba_golscasa + cuiaba_golstotal + xg_cuiaba_1410)/5
lambda_y = (cruzeiro_xgfora + cruzeiro_xgtotal + cruzeiro_golsfora + cruzeiro_golstotal + xg_cruzeiro_1410)/5

In [260]:
df_1410_madao = gerar_prob('Cuiaba', 'Cruzeiro', lambda_x, lambda_y)
df_1410_madao.to_excel("dados/previsoes/previsao1410-cuiabavscruzeiro_Modelo3.xlsx")